# 2.1 Individual Model 1-wenkanw data
## Hardware settings: 4cpus, 120GB,  1 gpus, gpu:V100, interconnection: 25ge


In [1]:
import numpy as np
import pandas as pd
import torch
from torch import optim, nn
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
from dataset import create_train_test_file_list, Person_MealsDataset, balance_data_indices

# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Device: ",device,"Device Count: ", torch.cuda.device_count(), "Device Name: ",torch.cuda.get_device_name()  )
    print("Torch version:", torch.__version__)
    
device

Device:  cuda Device Count:  2 Device Name:  Tesla V100-PCIE-16GB
Torch version: 1.7.1


device(type='cuda')

In [2]:
import sys

### imports
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Turn off TensorFlow logging
import tensorflow.compat.v1 as tf # maintain compatibility with TensorFlow 2.2.0

import keras
# from tensorflow.compat.v1.keras import backend as K # changed for compatibility with TensorFlow 2.2.0
import numpy as np
import pandas as pd
import random
#from sklearn.metrics import classification_report, confusion_matrix
from datetime import datetime
import loadfile
import addons

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, Activation
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D
from keras.utils import np_utils
from keras.regularizers import l1
from keras.models import load_model, save_model

from tensorflow.compat.v1.keras import backend as K
import numpy as np

shimmer_global_mean = [-0.012359981,-0.0051663737,0.011612018,
                        0.05796114,0.1477952,-0.034395125 ]

shimmer_global_stddev = [0.05756385,0.040893298,0.043825723,
                        17.199743,15.311142,21.229317 ]

shimmer_trended_mean = [-0.000002,-0.000002,-0.000000,
                0.058144,0.147621,-0.033260 ]

shimmer_trended_stddev = [0.037592,0.034135,0.032263,
                17.209038,15.321441,21.242532 ]

all_zero_means = [0,0,0,0,0,0]

meanvals = all_zero_means
stdvals = shimmer_trended_stddev


random_seed  = 1000
batch_size = 64

In [3]:
def print_settings(winmin,stridesec, EPOCHS):
    outfile = sys.stdout

    winlength = int(winmin * 60 * 15)
    step = int(stridesec * 15)
    start_time = datetime.now()
    arr = ["echo -n 'PBS: node is '; cat $PBS_NODEFILE",\
          "echo PBS: job identifier is $PBS_JOBID",\
          "echo PBS: job name is $PBS_JOBNAME"]

    [os.system(cmd) for cmd in arr]
    print("*****************************************************************\n", file=outfile, flush=True)
    print("Execution Started at " + start_time.strftime("%m/%d/%Y, %H:%M:%S"), file=outfile, flush=True)
    print("WindowLength: {:.2f} min ({:d} datum)\tSlide: {:d} ({:d} datum)\tEpochs:{:d}\n".format(winmin, winlength, stridesec, step, EPOCHS), file=outfile, flush=True)


In [4]:
EPOCHS = 30
winmin = 1 
stridesec = 1

print_settings(winmin,stridesec, EPOCHS)
# Load the dataset
meal_data_train = Person_MealsDataset(person_name= "wenkanw", file_name = "train_files", winmin = winmin,stridesec = stridesec)
meal_data_test = Person_MealsDataset(person_name= "wenkanw", file_name = "test_files", winmin = winmin,stridesec = stridesec)


*****************************************************************

Execution Started at 12/19/2020, 19:37:40
WindowLength: 1.00 min (900 datum)	Slide: 1 (15 datum)	Epochs:30

Loading Dataset ...
Loading File:  ../data/IndividualData/wenkanw-data/10-14-20/10-14-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-3-20/10-3-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-9-20/10-10-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-9-20/10-9-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-14-20/11-14-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-16-20/11-16-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-18-20/11-18-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-21-20/Dinner/Dinner.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-21-20/lunch/lunch.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-22-20/Dinner/Dinner.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-22-20/Lunch/L

In [5]:
#random.seed(random_seed)

# load
trainingsamples,traininglabels =  meal_data_train.data_indices, meal_data_train.labels
testsamples,testlabels =  meal_data_test.data_indices, meal_data_test.labels

train_shuffledUnderSampledBalancedIndices = balance_data_indices(traininglabels,mode="under", shuffle=True,random_state = random_seed,replace= False)
test_shuffledUnderSampledBalancedIndices = balance_data_indices(testlabels,mode="under", shuffle=True,random_state = random_seed,replace= False)
len(train_shuffledUnderSampledBalancedIndices),len(test_shuffledUnderSampledBalancedIndices)


(41878, 13520)

In [6]:
import collections
print([item for item, count in collections.Counter(train_shuffledUnderSampledBalancedIndices).items() if count > 1])


[]


In [7]:
# traininglabels[train_shuffledUnderSampledBalancedIndices]
testlabels[test_shuffledUnderSampledBalancedIndices]

array([1, 0, 0, ..., 0, 0, 1])

In [8]:
def split_train_test_indices(X, y, test_size, random_seed = None):
    """
    This function is to split the training set indices into validation set indices and training set indices
    
    X: indices of subset of the whole dataset
    y: labels of the whole dataset
    
    """
    from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, train_test_split

    train_indices = []
    test_indices = []
    if test_size ==0:
        train_indices = X
    elif test_size == 1:
        test_indices = X
    elif test_size >0 and test_size <1:
        labels = y[X]
        
        train_indices, test_indices, _,_ = train_test_split(X,labels ,
                                                            stratify=labels, 
                                                            test_size=test_size,random_state = random_seed)
        
    else:
        print("Invalid split ratio: %.3f"%(test_size))
    if len(train_indices)>0:
        print("Train set size: %d, with %d positive samples and %d negative samples"%(len(y[train_indices]),sum(y[train_indices]==1),
                                                                          sum(y[train_indices]==0)))
    if len(test_indices)>0:
        print("Test set size: %d, with %d positive samples and %d negative samples"%(len(y[test_indices]),
                                                                          sum(y[test_indices]==1),
                                                                           sum(y[test_indices]==0)))
    
    return train_indices, test_indices

In [9]:
train_indices, valid_indices = split_train_test_indices(X= train_shuffledUnderSampledBalancedIndices,
                                                        y = traininglabels, test_size = 0.2,
                                                       random_seed = random_seed)

Train set size: 33502, with 16751 positive samples and 16751 negative samples
Test set size: 8376, with 4188 positive samples and 4188 negative samples


In [10]:
a = set(train_indices)
b = set(valid_indices)
c = a.intersection(b)
c

set()

In [11]:
train_set_balanced = torch.utils.data.Subset(meal_data_train, train_indices)
valid_set_balanced = torch.utils.data.Subset(meal_data_train, valid_indices)

test_set_balanced = torch.utils.data.Subset(meal_data_test, test_shuffledUnderSampledBalancedIndices)

train_loader = torch.utils.data.DataLoader(train_set_balanced,batch_size=batch_size, shuffle=True,num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_set_balanced,batch_size=batch_size, shuffle=True,num_workers=2)

test_loader = torch.utils.data.DataLoader(test_set_balanced ,batch_size=batch_size, shuffle=True,num_workers=2)

print("Data Loader Created")

Data Loader Created


In [12]:
from model import *
from utils import *

torch.manual_seed(random_seed)
input_shape = (train_set_balanced[0][0].shape[0], train_set_balanced[0][0].shape[1])
model_4 = Discriminator_ResNet( ngpu=1, input_shape =input_shape , out_fea = 1)
# Since I use a dynamic created layer in network, need to input a sample to initialize the model first
_ = model_4(torch.rand((1, input_shape[0],input_shape[1])))
model_4.to(device)

criterion = nn.BCEWithLogitsLoss()

optimizer_4 = optim.Adam(model_4.parameters(),lr=0.01,  weight_decay=0.1)
lrscheduler_4 = optim.lr_scheduler.ReduceLROnPlateau(optimizer_4, mode='min',patience= 2, factor = 0.1,threshold=0.01)
dataloader = (train_loader, valid_loader)
model_4, best_model_4,val_score_4,loss_ls_4, train_acc_ls_4, valid_acc_ls_4 = train_model(model_4,dataloader, optimizer_4, 
                                                                    criterion, lrscheduler_4, device= device,
                                                                    n_epochs=30, patience = 5, l1_enabled=False,
                                                                    checkpoint_name ="../models/wenkanw_models/checkpoint_model_resnet.pt" )

Training set batch amounts: 524
Test set : 131
Start Training..
Epoch: 0,  Epoch_Loss: 0.4655, Train Acc: 80.4758 %, Train Recall: 0.7556 
Validation Acc:  82.1275 %,  Validation Recall: 0.8009 
Checkpoint Saved


Epoch: 1,  Epoch_Loss: 0.4356, Train Acc: 82.1772 %, Train Recall: 0.7726 
Validation Acc:  82.8558 %,  Validation Recall: 0.8665 
Checkpoint Saved


Epoch: 2,  Epoch_Loss: 0.4375, Train Acc: 81.8548 %, Train Recall: 0.7711 
Validation Acc:  82.3663 %,  Validation Recall: 0.7104 


Epoch: 3,  Epoch_Loss: 0.4359, Train Acc: 82.0250 %, Train Recall: 0.7686 
Validation Acc:  62.7507 %,  Validation Recall: 0.2569 


Epoch: 4,  Epoch_Loss: 0.4326, Train Acc: 82.2578 %, Train Recall: 0.7720 
Validation Acc:  83.0468 %,  Validation Recall: 0.7844 
Checkpoint Saved


Epoch: 5,  Epoch_Loss: 0.4364, Train Acc: 82.2070 %, Train Recall: 0.7735 
Validation Acc:  82.4379 %,  Validation Recall: 0.7151 


Epoch: 6,  Epoch_Loss: 0.4328, Train Acc: 82.1235 %, Train Recall: 0.7692 
Validation A

In [14]:
best_model_4.eval()
acc, recall = eval_model(best_model_4, test_loader,device)
print("Test Accuracy:", acc)

Test Accuracy: 81.25739644970415


In [16]:
%time acc, recall = eval_model(best_model_4, test_loader,device)

CPU times: user 1.77 s, sys: 283 ms, total: 2.06 s
Wall time: 6.08 s


In [18]:
%time acc, recall = eval_model(best_model_4.to("cpu"), test_loader,"cpu")

CPU times: user 29.4 s, sys: 13.4 s, total: 42.8 s
Wall time: 13.9 s
